<h1>概率图模型采样方法</h1>

> <b>解决问题</b>

> 从数据分布中采样，尤其是高维数据分布

参考文献：

[1] Pattern recognition and machine learning, Chapter 11.

[2] Machine Learning, A Probabilistic Perspective, Chapter 23, 24.

# 1. 标准分布直接采样 （Sampling from Standard Distribution）

>定理: $F$是我们要采样的函数$p(x)$的累积密度函数(cdf)，如果$u\sim U(0, 1)$， 则$F^{-1}(u)\sim F$.
>证明见MLAPP.

采样过程：

（1）采样$u\sim U(0, 1)$；

（2）计算$x=F^{-1}(u)$。

直接采样需要计算累积密度函数的逆函数，如果逆函数难以计算，则该方法不适用。




# 2. 拒绝采样 （Rejection Sampling）

如果待采样分布$p(x)$难以直接采样，建立一个参考分布$q(x)$，使得对于某些固定的参数$M$，满足$Mq(x)\geq \tilde{p}(x)$。其中$\tilde{p}(x)$是$p(x)$的一个未归一化的版本，例如，$p(x)=\tilde{p}(x)/Z_{p}$，$Z_{p}$是一个固定的归一化参数。则函数$Mq(x)$是$\tilde{p}(x)$的一个上界包围。

采样过程：

（1）采样$x\sim q(x)$；

（2）采样$u\sim U(0,1)$；

（3）接受率$r=\frac{\tilde{p}(x)}{Mq(x)}$，如果$u<r$，接受本次采样，否则拒绝；

拒绝采样的$M$不好设置，设置的过小，$q(x)$无法全部包围住$p(x)$；设置的过大，则接受率太低，采样效率太低。也要求$q(x)$既便于采样，又要从形状上比较接近$p(x)$，在高维情况或者$p(x)$很复杂的情况下难以满足。
